In [141]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import balanced_accuracy_score

In [142]:
df = pd.read_csv('all_seasons.csv')
df3 = pd.read_csv('all_seasons.csv')
a= ['player_name', 'age', 'player_height', 'player_weight', 'pts', 'reb']
df = df[a]
df.head()

,player_name,age,player_height,player_weight,pts,reb
0,Chris Robinson,23.0,195.58,90.71840,4.6,1.7
1,Matt Fish,27.0,210.82,106.59412,0.3,0.8
2,Matt Bullard,30.0,208.28,106.59412,4.5,1.6
3,Marty Conlon,29.0,210.82,111.13004,7.8,4.4
4,Martin Muursepp,22.0,205.74,106.59412,3.7,1.6


### Testing

In [143]:
df['age <= 25'] = df.apply(lambda row: 1 if row['age'] <= 25 else 0,axis=1)
df['player_height >= 180'] = df.apply(lambda row: 1 if row['age'] >= 180 else 0,axis=1)
df['player_weight <= 90'] = df.apply(lambda row: 1 if row['age'] <= 90 else 0,axis=1)
df['pts >= 6'] = df.apply(lambda row: 1 if row['age'] >= 6 else 0,axis=1)
df['reb >= 3'] = df.apply(lambda row: 1 if row['age'] >= 3 else 0,axis=1)
df['Cocok'] = df.apply(lambda row: 'ya' if (row['age'] <= 25)&(row['player_height'] >= 180)&(row['player_weight']<=80)&(row['pts'] >=6)&(row['reb'] >=3) else 'tidak',axis=1)
df.head()

,player_name,age,player_height,player_weight,pts,reb,age <= 25,player_height >= 180,player_weight <= 90,pts >= 6,reb >= 3,Cocok
0,Chris Robinson,23.0,195.58,90.71840,4.6,1.7,1,0,1,1,1,tidak
1,Matt Fish,27.0,210.82,106.59412,0.3,0.8,0,0,1,1,1,tidak
2,Matt Bullard,30.0,208.28,106.59412,4.5,1.6,0,0,1,1,1,tidak
3,Marty Conlon,29.0,210.82,111.13004,7.8,4.4,0,0,1,1,1,tidak
4,Martin Muursepp,22.0,205.74,106.59412,3.7,1.6,1,0,1,1,1,tidak


In [144]:
df.isnull().sum()

player_name             0
age                     0
player_height           0
player_weight           0
pts                     0
reb                     0
age <= 25               0
player_height >= 180    0
player_weight <= 90     0
pts >= 6                0
reb >= 3                0
Cocok                   0
dtype: int64

## Standarisasi

In [148]:
a = StandardScaler().fit_transform(df[["age"]])
b = StandardScaler().fit_transform(df[["player_height"]])
c = StandardScaler().fit_transform(df[["player_weight"]])
d = StandardScaler().fit_transform(df[["pts"]])  
e = StandardScaler().fit_transform(df[["reb"]])
df3["ageskl"] = a
df3["player_heightskl"] = b
df3["player_weight"] = c
df3['pts'] = d
df3['reb'] = e
df3.head()

,Unnamed: 0,player_name,team_abbreviation,age,player_height,player_weight,college,country,draft_year,draft_round,...,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season,ageskl,player_heightskl
0,0,Chris Robinson,VAN,23.0,195.58,-0.802604,Western Kentucky,USA,1996,2,...,1.6,-11.4,0.039,0.088,0.155,0.486,0.156,1996-97,-0.996742,-0.578993
1,1,Matt Fish,MIA,27.0,210.82,0.439607,North Carolina-Wilmington,USA,1992,2,...,0.0,-15.1,0.143,0.267,0.265,0.333,0.000,1996-97,-0.077272,1.063130
2,2,Matt Bullard,HOU,30.0,208.28,0.439607,Iowa,USA,Undrafted,Undrafted,...,0.9,0.9,0.016,0.115,0.151,0.535,0.099,1996-97,0.612331,0.789443
3,3,Marty Conlon,BOS,29.0,210.82,0.794524,Providence,USA,Undrafted,Undrafted,...,1.4,-9.0,0.083,0.152,0.167,0.542,0.101,1996-97,0.382463,1.063130
4,4,Martin Muursepp,DAL,22.0,205.74,0.439607,None,USA,1996,1,...,0.5,-14.5,0.109,0.118,0.233,0.482,0.114,1996-97,-1.226609,0.515755


In [149]:
rekrut = []
for x in range(len(df.values)):               
    if df['age'][x] <= 25 and df['player_height'][x] >= 180 and df['player_weight'][x] <= 90 and df['pts'][x] >= 6 and df['reb'][x] >= 3:
        rekrut.append('patut direkrut')
    else:
        rekrut.append('tidak direkomendasikan')
df['Recruit'] = rekrut

### Splitting

In [150]:
train = df[['age','player_height','player_weight','pts','reb','Recruit']]
dfTrain = pd.DataFrame(train)

xtr,xts,ytr,yts = train_test_split(df[['age','player_height','player_weight','pts','reb']],df['Recruit'],test_size = .18)

def n_values():
    n = round((len(xtr) + len(df[['age','player_height','player_weight','pts','reb']])) ** .5)
    if (n % 2 == 0):
        return n +1
    else:
        return n

xtr

,age,player_height,player_weight,pts,reb
2854,22.0,213.36,102.965384,19.0,8.8
4938,22.0,205.74,95.254320,7.9,5.0
2243,23.0,203.20,98.883056,8.3,6.0
9264,39.0,200.66,106.594120,3.2,1.9
8690,23.0,200.66,83.914520,1.2,0.3
...,...,...,...,...,...
5600,32.0,203.20,108.862080,3.0,2.2
5596,31.0,210.82,111.130040,4.0,3.8
2986,32.0,203.20,106.594120,4.5,4.4
574,26.0,182.88,77.110640,5.1,1.4


In [151]:
modelLog = LogisticRegression(solver='lbfgs')
modelSVC = SVC(gamma='auto', probability=True)
modelTR = tree.DecisionTreeClassifier()
modelRF = RandomForestClassifier(n_estimators=100)
modelKN = KNeighborsClassifier(n_neighbors=n_values())

-----------------------------------------

## Logistic Regression

In [152]:
modelLog = LogisticRegression(solver='lbfgs', multi_class='auto',max_iter=100000)
modelLog.fit(xts,yts)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

------------------------------

## K-Neighbours

In [153]:
def nilai_k():
    k = round((len(df['Cocok'])) ** .5)
    if k % 2 == 0:
        return k + 1
    else:
        return k
modelKN = KNeighborsClassifier(
    n_neighbors=nilai_k()
)
modelKN .fit(xtr, ytr)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=99, p=2,
                     weights='uniform')

### Random Forest

In [154]:
modelRF= RandomForestClassifier(n_estimators=100)
modelRF.fit(df[['age','player_height','player_weight','pts','reb']],df['Recruit'])

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

## K-Fold Cross Validation

In [155]:
from sklearn.model_selection import cross_val_score
print(cross_val_score(LogisticRegression(solver='lbfgs'),xtr,ytr,cv = 5).mean())

0.9826530612244898


In [156]:
print(np.mean(cross_val_score(SVC(gamma='auto'), xtr,ytr,cv = 5)))

0.9841836734693878


In [157]:
# print(cross_val_score(tree.DecisionTreeClassifier(),xtr,ytr,cv = 5).mean())

In [158]:
print(cross_val_score(KNeighborsClassifier(n_neighbors=n_values()),xtr,ytr,cv = 5).mean())

0.9811224489795919


In [159]:
print(cross_val_score(RandomForestClassifier(n_estimators=100),xtr,ytr,cv = 5).mean())

0.999234693877551


------------------------------------------------

### Evaluation Metrics

In [160]:
ypred = modelRF.predict(xts)
print(balanced_accuracy_score(yts, ypred))
print(precision_score(yts, ypred))
print(recall_score(yts, ypred))
print(f1_score(yts, ypred))
print(roc_auc_score(yts, ypred))
print(ypred[:10])
print(yts[:10])

1.0


C:\Users\ADAM-PC\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1305: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if pos_label not in present_labels:


ValueError: pos_label=1 is not a valid label: array(['patut direkrut', 'tidak direkomendasikan'], dtype='<U22')

In [161]:
ypred = modelKN.predict(xts)
print(balanced_accuracy_score(yts, ypred))
print(precision_score(yts, ypred))
print(recall_score(yts, ypred))
print(f1_score(yts, ypred))
print(roc_auc_score(yts, ypred))
print(ypred[:10])
print(yts[:10])

0.625


C:\Users\ADAM-PC\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1305: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if pos_label not in present_labels:


ValueError: pos_label=1 is not a valid label: array(['patut direkrut', 'tidak direkomendasikan'], dtype='<U22')

In [162]:
ypred = modelLog.predict(xts)
print(balanced_accuracy_score(yts, ypred))
print(precision_score(yts, ypred))
print(recall_score(yts, ypred))
print(f1_score(yts, ypred))
print(roc_auc_score(yts, ypred))
print(ypred[:10])
print(yts[:10])

0.6229277679100059


C:\Users\ADAM-PC\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1305: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if pos_label not in present_labels:


ValueError: pos_label=1 is not a valid label: array(['patut direkrut', 'tidak direkomendasikan'], dtype='<U22')

## Data Prediksi

In [163]:
data = {'nama': ['Andakara Prastawa Dyaksa','Reggie Mononimbar','Hardianus Lakudu','Kevin Yonas Sitorus','Arki Dikania Wisnu','Laurentius Steven Oei','Mei Joni','Vincent Rivaldi Kosasih','Hardian Wicaksono','Brandon Jawato'],
        'club': ['Pelita Jaya Bakrie', 'Pelita Jaya Bakrie', 'Satria Muda Pertamina Jakarta', 'Satria Muda Pertamina Jakarta', 'Satria Muda Pertamina Jakarta', 'Satria Muda Pertamina Jakarta', 'Stapac','Stapac', 'Pacific Caesar Surabaya', 'Louvre Surabaya'],
        'country' : ['Indonesia','Indonesia','Indonesia','Indonesia','Indonesia','Indonesia','Indonesia','Indonesia','Indonesia','Indonesia'],
        'age': [24, 21, 23, 26, 20, 21, 25, 23, 21, 24],
        'player_height':[190, 185, 178, 185, 183, 191, 188, 179, 177, 182],
        'player_weight': [90,86, 83, 75, 80, 85, 90, 87, 80, 85],
        'pts' :[7,6,10,11,5,4,7,1,9,6],
        'reb' : [6,3,3,4,2,10,5,2,8,5]
       }

In [164]:
df2 = pd.DataFrame(data)
df2

,nama,club,country,age,player_height,player_weight,pts,reb
0,Andakara Prastawa Dyaksa,Pelita Jaya Bakrie,Indonesia,24,190,90,7,6
1,Reggie Mononimbar,Pelita Jaya Bakrie,Indonesia,21,185,86,6,3
2,Hardianus Lakudu,Satria Muda Pertamina Jakarta,Indonesia,23,178,83,10,3
3,Kevin Yonas Sitorus,Satria Muda Pertamina Jakarta,Indonesia,26,185,75,11,4
4,Arki Dikania Wisnu,Satria Muda Pertamina Jakarta,Indonesia,20,183,80,5,2
5,Laurentius Steven Oei,Satria Muda Pertamina Jakarta,Indonesia,21,191,85,4,10
6,Mei Joni,Stapac,Indonesia,25,188,90,7,5
7,Vincent Rivaldi Kosasih,Stapac,Indonesia,23,179,87,1,2
8,Hardian Wicaksono,Pacific Caesar Surabaya,Indonesia,21,177,80,9,8
9,Brandon Jawato,Louvre Surabaya,Indonesia,24,182,85,6,5


----------------------

### Prediksi

Prediksi Random Forest

In [165]:
df2['rekomendasi']=modelRF.predict(df2[['age','player_height','player_weight','pts','reb']])
df2

,nama,club,country,age,player_height,player_weight,pts,reb,rekomendasi
0,Andakara Prastawa Dyaksa,Pelita Jaya Bakrie,Indonesia,24,190,90,7,6,patut direkrut
1,Reggie Mononimbar,Pelita Jaya Bakrie,Indonesia,21,185,86,6,3,tidak direkomendasikan
2,Hardianus Lakudu,Satria Muda Pertamina Jakarta,Indonesia,23,178,83,10,3,tidak direkomendasikan
3,Kevin Yonas Sitorus,Satria Muda Pertamina Jakarta,Indonesia,26,185,75,11,4,tidak direkomendasikan
4,Arki Dikania Wisnu,Satria Muda Pertamina Jakarta,Indonesia,20,183,80,5,2,tidak direkomendasikan
5,Laurentius Steven Oei,Satria Muda Pertamina Jakarta,Indonesia,21,191,85,4,10,tidak direkomendasikan
6,Mei Joni,Stapac,Indonesia,25,188,90,7,5,patut direkrut
7,Vincent Rivaldi Kosasih,Stapac,Indonesia,23,179,87,1,2,tidak direkomendasikan
8,Hardian Wicaksono,Pacific Caesar Surabaya,Indonesia,21,177,80,9,8,tidak direkomendasikan
9,Brandon Jawato,Louvre Surabaya,Indonesia,24,182,85,6,5,tidak direkomendasikan


In [169]:
# df2['rekomendasi']=Logreg.predict(df2[['age','player_height','player_weight','pts','reb']])
# df2

In [168]:
# df2['rekomendasi']=KNN.predict(df2[['age','player_height','player_weight','pts','reb']])
# df2

In [ ]:
# 